We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.

In [ ]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"): 
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")


In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.fastembed import FastEmbedEmbeddings

from langchain_iris import IRISVector


In [ ]:
loader = TextLoader("../data/state_of_the_union.txt", encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
# embeddings = FastEmbedEmbeddings()

In [ ]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [ ]:
print(CONNECTION_STRING)

In [ ]:
COLLECTION_NAME = "state_of_the_union_test"

#db = IRISVector.from_documents(
    #embedding=embeddings,
    #documents=docs,
    #collection_name=COLLECTION_NAME,
    #connection_string=CONNECTION_STRING,
#)

# If reconnecting to the database, use this:

db = IRISVector(
     embedding_function=embeddings,
     dimension=1536,
     collection_name=COLLECTION_NAME,
     connection_string=CONNECTION_STRING,
 )

In [ ]:
# To add documents to existing vector store:

# db.add_documents(docs)

In [ ]:
print(f"Number of docs in vector store: {len(db.get()['ids'])}")

In [ ]:
query = "Joint patrols to catch traffickers"
docs_with_score = db.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

In [ ]:
db.add_documents([Document(page_content="foo")])
docs_with_score = db.similarity_search_with_score("foo")
docs_with_score[0]

In [ ]:
docs_with_score

In [ ]:
retriever = db.as_retriever()
print(retriever)